## Scratch_learning

Here, I employ **PatchTST**, one of the SOTA transformer models for time series classification, using a **scratch learning strategy**. To begin, I first need to upload the series of **transformers** modules from **Hugging Face**😊.

In [1]:
from data_processer import *
import torch
from transformers import (
    PatchTSTConfig, 
    PatchTSTForClassification,
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
 )

2025-03-15 17:30:17.749832: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/ubuntu/anaconda3/envs/gpt/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/ubuntu/anaconda3/envs/g

Let's load the data (20nm) by using **TS_Processsor** I wrote, this module transforms the original data to the right input form that model could received, including normolizing, slicing, outlier removal, and generating Dataset.

In [2]:
data_path = "data_20nm"
ts_processor = TS_Processor(sequence_length=1024, stride=64, remove_extreme_values=True, iqr_threshold=7.0)
dataset_dict = ts_processor(data_path)

print(dataset_dict)

Detected and removed outliers:
File: degree_80.csv, Index: 0, Data: {'time': '2024-12-04 18:15:41.860.', '2-1[m/s^2]': 0.2266, '2-2[m/s^2]': 990.13763, '2-3[m/s^2]': -0.05433, '2-4[m/s^2]': 1016.1023, '2-5[m/s^2]': -1003.1174, '2-6[m/s^2]': 0.98538}
File: degree_60.csv, Index: 0, Data: {'time': '2024-12-04 17:03:32.427.', '2-1[m/s^2]': -32.35963, '2-2[m/s^2]': 1.6487, '2-3[m/s^2]': 1017.5576, '2-4[m/s^2]': 1015.3014, '2-5[m/s^2]': 1004.6065, '2-6[m/s^2]': -0.18368}
File: degree_60.csv, Index: 217296, Data: {'time': '2024-12-04 17:03:43.292.', '2-1[m/s^2]': -23.24958, '2-2[m/s^2]': -6.59387, '2-3[m/s^2]': 5.18964, '2-4[m/s^2]': -5.53556, '2-5[m/s^2]': -3.99335, '2-6[m/s^2]': -17.72532}
File: degree_40.csv, Index: 0, Data: {'time': '2024-12-04 16:14:39.579.', '2-1[m/s^2]': 29.76894, '2-2[m/s^2]': -990.73804, '2-3[m/s^2]': 2.5759, '2-4[m/s^2]': 1015.4796, '2-5[m/s^2]': 0.23558, '2-6[m/s^2]': 0.02296}
DatasetDict({
    train: Dataset({
        features: ['sequence', 'label'],
        num_r

Here is the configuration and model init!

In [3]:
config = PatchTSTConfig(
    num_input_channels=len(dataset_dict["train"][0]["sequence"][0]), 
    num_targets=len(set(dataset_dict["train"]["label"])), 
    context_length=len(dataset_dict["train"][0]["sequence"]), 
    patch_length=16,
    stride=16,
    use_cls_token=True,
    num_hidden_layers=3,
    d_model=128,
    num_attention_heads=4,
    ffn_dim=512
)
model = PatchTSTForClassification(config=config)

training_args = TrainingArguments(
    output_dir="./results_20nm",
    evaluation_strategy="steps", 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100,
    load_best_model_at_end=True, 
    metric_for_best_model="eval_loss", 
  #  greater_is_better=True, 
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    label_names=["target_values"],
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

/home/ubuntu/anaconda3/envs/gpt/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Let's begin !!!!!!

In [4]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,1.090300,1.069090,0.342356
200,0.675600,0.540018,0.741039
300,0.483400,0.384006,0.843819
400,0.400200,0.350763,0.854426
500,0.362400,0.340539,0.855157
600,0.300600,0.262873,0.890636
700,0.291600,0.219991,0.915508
800,0.263000,0.226372,0.908925
900,0.234000,0.203744,0.922458
1000,0.233800,0.224462,0.909656


TrainOutput(global_step=3400, training_loss=0.21064802495872273, metrics={'train_runtime': 2530.4332, 'train_samples_per_second': 76.742, 'train_steps_per_second': 2.399, 'total_flos': 2922028791091200.0, 'train_loss': 0.21064802495872273, 'epoch': 5.601317957166392})

As we can see the  'eval_accuracy' is up to 96%, what a nice result! hahaha!

In [5]:
test_results = trainer.evaluate(eval_dataset=dataset_dict["test"])
print(test_results)

{'eval_loss': 0.10185562819242477, 'eval_accuracy': 0.9601160261058739, 'eval_runtime': 47.4063, 'eval_samples_per_second': 116.356, 'eval_steps_per_second': 3.649, 'epoch': 5.601317957166392}
